<a href="https://colab.research.google.com/github/charleszhang418/SpaceX/blob/main/code/dnabert_finetune_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

+ Data: https://osdr.nasa.gov/bio/repo/data/studies/OSD-466
+ Sample:
  1. RR10_FCS_FLT_KO_F19: p21-null, Space Flight
  2. RR10_FCS_FLT_WT_F16: Wild Type, Space Flight
  3. RR10_FCS_GC_WT_G3: Wild Type, Ground Control
  4. RR10_FCS_GC_KO_G4: p21-null, Ground Control
  5. RR10_FCS_VIV_WT_V1: Wild Type, Vivarium Control
  6. RR10_FCS_VIV_KO_V13: p21-null,	Vivarium Control

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# %cd gdrive/MyDrive/NASA
%cd gdrive/MyDrive/Project/data_science/nasa-space-app-2023/
!ls

/content/gdrive/MyDrive/Project/data_science/nasa-space-app-2023
code  data  logs  results


In [3]:
!pip install transformers
!pip install torch
!pip install einops
!pip install transformers[torch]
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 10.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, BertModel
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import evaluate
import numpy as np

In [5]:
dna_data = pd.read_csv('data/dna_data.csv')
print(dna_data.shape)
dna_data.head()

(1250, 3)


,notation,DNA,mass
0,SRR5756263.1 1 length=150,NTTATATAAGATAAAGGTTTTAATTGTATTAAGTGATAGTTAAGTC...,"<AAAKKKKAAKKKAFKAFFAKKKKAKKKF,FF,AKFF,,FFKKKKF..."
1,SRR5756263.2 2 length=150,NTTTTAAATGTGAATATATTTGATTTTTTTTGGTTAATTTGATTTG...,"AAFFFKKKKFKKKKKKKKKKKKKFFKKKK7AFAKKKK,FKKKKKKK..."
2,SRR5756263.3 3 length=150,NAATTAAAATTATAAATATTGGAATTAAGAATATTTAAGTGTTCGG...,"AAFFKKKKKKKKKKAKAKKKFFAKFKKKKKKKAKKKFK7KKKKK,,..."
3,SRR5756263.4 4 length=150,NATTATAATTTATTGTAGTATTTTGGAGAAATGTGTTATTATTGAA...,AAF<FKKKKKKKKKKKKKAKFKKKAFKKKKKA77FKKKKKKKKKKK...
4,SRR5756263.5 5 length=150,NTAGGAGTGTTGGTAGATGTGATATAAATTAAAGGCATTATTATTA...,"AAFFFK,F<FFFFKKKKKAFKKKKKKKKKFKKKK,AAFKFFKKFKA..."


In [6]:
tokenizer = AutoTokenizer.from_pretrained('zhihan1996/DNABERT-2-117M', trust_remote_code=True)
# model = BertModel.from_pretrained('zhihan1996/DNABERT-2-117M', trust_remote_code=True)
model = BertForSequenceClassification.from_pretrained('zhihan1996/DNABERT-2-117M', num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.encoder.layer.1.attention.self.value.bias', 'bert.encoder.layer.7.attention.self.value.weight', 'bert.encoder.layer.8.output.dense.bias', 'bert.embeddings.position_embeddings.weight', 'bert.encoder.layer.2.attention.self.key.weight', 'bert.encoder.layer.2.intermediate.dense.bias', 'bert.encoder.layer.10.output.LayerNorm.weight', 'bert.encoder.layer.7.attention.self.key.bias', 'bert.encoder.layer.7.attention.self.query.weight', 'bert.encoder.layer.11.attention.self.key.weight', 'bert.encoder.layer.6.intermediate.dense.bias', 'bert.encoder.layer.2.output.dense.bias', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.2.attention.self.query.bias', 'bert.encoder.layer.6.attention.self.query.weight', 'bert.encoder.layer.3.attention.self.key.weight', 'bert.encoder.layer.8.attention.self.query.weight', 'bert.encoder.laye

In [7]:
# Create random label for testing
import random
N = len(dna_data)
random_label_list = [random.randint(0, 2) for _ in range(N)]
dna_data['label'] = random_label_list

In [22]:
train_data = dna_data.sample(frac=0.8, random_state=524)
test_eval_data = dna_data.drop(train_data.index)
eval_data = test_eval_data.sample(frac=0.5, random_state=524)
test_data = test_eval_data.drop(eval_data.index)

print(train_data.shape, eval_data.shape, test_data.shape)

train_dna = list(train_data['DNA'])
train_labels = list(train_data['label'])

val_dna = list(eval_data['DNA'])
val_labels = list(eval_data['label'])


test_dna = list(test_data['DNA'])
test_labels = list(test_data['label'])

train_encodings = tokenizer(train_dna, truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(val_dna, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_dna, truncation=True, padding=True, return_tensors='pt')

(1000, 4) (125, 4) (125, 4)


In [23]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [24]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  acc = metric.compute(predictions=predictions, references=labels)['accuracy']
  return {
      'accuracy': acc
  }

training_args = TrainingArguments(
    output_dir='results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    learning_rate=2e-5,
    logging_dir='logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(test_dataset)

# model.save_pretrained("./your_finetuned_model")
# tokenizer.save_pretrained("./your_finetuned_tokenizer")

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.162263,0.328000
2,No log,2.307939,0.312000
3,No log,1.677833,0.288000
4,No log,2.060024,0.296000
5,No log,2.121611,0.328000
6,No log,2.216405,0.312000
7,No log,2.127733,0.312000
8,0.623600,2.331011,0.328000
9,0.623600,2.664074,0.312000
10,0.623600,2.700023,0.312000


In [25]:
results

{'eval_loss': 2.5745129585266113,
 'eval_accuracy': 0.336,
 'eval_runtime': 0.199,
 'eval_samples_per_second': 628.021,
 'eval_steps_per_second': 40.193,
 'epoch': 10.0}